### Multi-class classification | Outline

1. Load the data
2. Explore and plot the data
3. Transform the data
4. Train the model
5. Evaluate the model
6. Saving the model

In [1]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML,1.4.0-preview"  
    
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;

Installing package Microsoft.ML, version 1.4.0-preview....................................done!

Successfully added reference to package Microsoft.ML, version 1.4.0-preview

Installing package XPlot.Plotly, version 2.0.0...........done!

Successfully added reference to package XPlot.Plotly, version 2.0.0

In [5]:
using Microsoft.ML.Data;

public class ProductInformation 
{   
    [LoadColumn(0)]
    public string ProductName;
       
    [LoadColumn(1)]
    public string Category;     
    
    [LoadColumn(2)]
    public float Price;
    
    [LoadColumn(3)]
    public string Description;   
    
    [LoadColumn(4)]
    public string Brand;   
}

In [6]:
var dataPath = "./Datasets/Multi-Class-Classification Category/flipkart_com-ecommerce_sample.csv";

var mlContext = new MLContext(seed: 1);

IDataView dataView = mlContext.Data.LoadFromTextFile<ProductInformation>(dataPath, hasHeader: true, separatorChar: ',');

#### Data exploration
1. Dump DataFrame
2. Missing values?
3. Plot histogram of possible values for product categories
4. Reviews over category?

In [7]:
var rows = mlContext.Data.CreateEnumerable<ProductInformation>(dataView, reuseRowObject: false)
                    .Take(45)
                    .ToList();

display(rows);

index ProductName Category Price Description Brand 0 Alisha Solid Women's Cycling Shorts Clothing 999 Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy Red NavySpecifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts Alisha 1 FabHomeDecor Fabric Double Sofa Bed Furniture 32157 FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Price: Rs. 22646 • Fine deep seating experience • Save Space with the all new click clack Sofa Bed • Easy to fold and vice versa with simple click clack mechanism • Chrome legs with mango wood frame for long term durability • Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience • A double bed that can easily sleep twoSpecifications of FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contemporary & Modern Filling Material Microfiber Seating Capacity 3 Seater Upholstery Type NA Upholstery Included No Bed Size Double Shape Square Suitable For Living Room Model Number FHD112 Care Instructions Avoid outdoor use and exposure to water or prolonged moisture Avoid exposure to direct heat or sunlight as this can cause the sofa colour to fade Keep sharp objects away from your sofa A little tear on the fabric cover may be hard to repair Vacuum your sofas periodically with a soft bristled bru...View More Avoid outdoor use and exposure to water or prolonged moisture Avoid exposure to direct heat or sunlight as this can cause the sofa colour to fade Keep sharp objects away from your sofa A little tear on the fabric cover may be hard to repair Vacuum your sofas periodically with a soft bristled brush attachment or lightly brush them to keep general dirt and dust off the sofa and prevent any embedding between the fibres Try to avoid food and drink spillage of any kind If spills occur do not leave unattended In case of a stain a water-free fabric cleaner can be used However avoid applying the cleaner directly on the stain as this can cause damage to the fabric and fade colour Pour the cleaner onto a clean cloth and test its effect on a hidden area of the sofa before cleaning the stain with the cloth A professional scotchguard treatment is one of the easiest and most effective options to protect against spills or stains and keep pet hair at bay Getting your sofa professionally cleaned once every 6-8 months will not only take care of the nooks and corners that you can't reach it will also make it more durable Finish Type Matte Important Note Cancellation NOT allowed for this product after 24 hrs of order booking. Warranty Covered in Warranty Warranty covers all kind of manufacturing defects. Concerned product will either be repaired or replaced based on discretion. Service Type Manufacturer Warranty Warranty Summary 6 Months Domestic Warranty Not Covered in Warranty Warranty does not cover for Improper Handling Dimensions Weight 40 kg Height 838 mm Width 1905 mm Depth 939 mm Disclaimer - The color of the product may vary slightly compared to the picture displayed on your screen. This is due to lighting pixel quality and color settings - Please check the product's dimensions to ensure the product will fit in the desired location. Also check if the product will fit through...View More - The color of the product 

#### Plotting

In [81]:
var categories = dataView.GetColumn<string>("Category");

var categoriesHistogram = Chart.Plot(new Graph.Histogram 
    {
        x = categories,  
    });

categoriesHistogram.WithLayout(new Layout.Layout { title="Distribution of categories" });
display(categoriesHistogram);

#### Split our data

In [30]:
var trainTestSplit = mlContext.Data.TrainTestSplit(dataView);

#### Data Transformation

In [39]:
var dataProcessPipeline = mlContext.Transforms.Conversion.MapValueToKey("Category", "Category")
                              .Append(mlContext.Transforms.Categorical.OneHotHashEncoding("ProductName"))
                              .Append(mlContext.Transforms.Categorical.OneHotHashEncoding("Brand"))
                              .Append(mlContext.Transforms.Text.FeaturizeText("Description"))
                              .Append(mlContext.Transforms.Concatenate("Features", new[] { "ProductName", "Brand", "Description" }))
                              .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"));

In [50]:
var trainer = mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy("Category", "Features")
                .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel", "PredictedLabel"));

var trainingPipeline = dataProcessPipeline.Append(trainer);

##### Train the model

In [51]:
ITransformer model = trainingPipeline.Fit(trainTestSplit.TrainSet);

#### Evaluate the model

In [217]:
var predicitions = model.Transform(trainTestSplit.TestSet);
var metrics = mlContext.MulticlassClassification.Evaluate(predicitions, labelColumnName: "Category");

display("Accuracy:" + " " + metrics.MicroAccuracy)

Accuracy: 0.9795918367346939

Microsoft.DotNet.Interactive.DisplayedValue

In [208]:
var confusionMatrix = Chart.Plot(new Graph.Heatmap 
    {
        x = categories.Distinct(),
        y = categories.Distinct().Reverse(),
        z = metrics.ConfusionMatrix.Counts.Reverse(),   
        zmin = 0,
        zmax = 5,
        colorscale = "off"
    });

confusionMatrix.WithLayout(new Layout.Layout { title="Confusion Matrix" });
confusionMatrix

#### Save the model

In [54]:
mlContext.Model.Save(model, trainTestSplit.TrainSet.Schema, "productCategoryClassifier.zip");